<a href="https://colab.research.google.com/github/windyday0622/windyday/blob/main/AI%20Agent%20%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8%20%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%83%9D%EC%84%B1%20%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8%20/4_AI_%EA%B8%B0%EB%B0%98_%EC%84%9C%EB%B9%84%EC%8A%A4_%EB%8C%80%EC%9D%91_%EB%A9%94%EC%8B%9C%EC%A7%80_%EC%83%9D%EC%84%B1_%EB%8F%84%EA%B5%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI 기반 서비스 대응 메시지 생성 도구를 개발하는 과정은 여러 단계로 나눠지며, 고객 서비스 자동화와 관련된 문제를 해결하는 데 중점을 둡니다.

1. 기본 데이터 수집
고객 서비스에서 사용하는 실제 메시지 데이터를 수집해야 합니다. 이러한 데이터는 고객 질문, 불만, 칭찬 등 다양한 유형을 포함하고 있어야 합니다.

데이터 출처: 고객 서비스 기록, FAQ, 고객 피드백 등에서 데이터를 수집할 수 있습니다.
데이터 예시:
고객의 질문: "이 제품은 어떻게 사용하나요?"
고객의 불만: "배송이 너무 늦었어요."
고객의 칭찬: "이 서비스 정말 좋네요!"

In [1]:
import pandas as pd

# 예시 고객 서비스 데이터
data = {
    'message': [
        "이 제품은 어떻게 사용하나요?",
        "배송이 너무 늦었어요.",
        "이 서비스 정말 좋네요!",
        "제품에 하자가 있었어요.",
        "가격이 비싸요.",
    ],
    'type': ['질문', '불만', '칭찬', '불만', '불만']
}

# 데이터프레임으로 변환
df = pd.DataFrame(data)
print(df)


            message type
0  이 제품은 어떻게 사용하나요?   질문
1      배송이 너무 늦었어요.   불만
2     이 서비스 정말 좋네요!   칭찬
3     제품에 하자가 있었어요.   불만
4          가격이 비싸요.   불만


2. 대응 메시지 유형 분류
고객 서비스 메시지는 질문, 불만, 칭찬 등으로 분류해야 합니다. 이를 위해 텍스트 분류 모델을 사용할 수 있습니다.

모델 선택: 텍스트 분류를 위한 기존의 사전 학습된 모델을 사용할 수 있습니다. 예를 들어, BERT나 DistilBERT와 같은 모델을 활용하여 문장을 분류할 수 있습니다.

In [2]:
from sklearn.model_selection import train_test_split
from transformers import pipeline, BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report

# 예시로 사용하는 텍스트 분류 모델
model_name = 'bert-base-uncased'  # BERT 사전 학습된 모델
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 사전 학습된 텍스트 분류 파이프라인 사용
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# 예시 데이터 분류
messages = df['message'].tolist()
predictions = classifier(messages)

# 결과 확인
print(predictions)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.6351618766784668}, {'label': 'LABEL_1', 'score': 0.6445944905281067}, {'label': 'LABEL_1', 'score': 0.6114877462387085}, {'label': 'LABEL_1', 'score': 0.6527814865112305}, {'label': 'LABEL_1', 'score': 0.652129054069519}]


3. 언어 모델 훈련 (Fine-tuning)
이제 Fine-tuning을 통해 모델을 특정 도메인에 맞게 훈련시켜야 합니다. 이 단계에서는 실제 고객 서비스 데이터를 사용하여 모델을 훈련시킵니다.

Fine-tuning을 위한 데이터 준비:
고객 서비스 메시지와 그에 맞는 대응 메시지 데이터를 포함한 데이터셋이 필요합니다. 예를 들어, 질문에 대한 답변, 불만에 대한 사과 메시지, 칭찬에 대한 감사 메시지를 포함한 데이터셋을 사용합니다

In [3]:
# Fine-tuning 데이터 예시 (질문, 불만, 칭찬에 대한 대응)
fine_tuning_data = {
    'input': [
        "이 제품은 어떻게 사용하나요?",
        "배송이 너무 늦었어요.",
        "이 서비스 정말 좋네요!"
    ],
    'output': [
        "제품 사용법에 대한 정보를 제공해 드리겠습니다.",
        "배송 지연에 대해 사과드립니다. 빠르게 처리하도록 하겠습니다.",
        "고객님께서 만족하셔서 기쁩니다! 감사합니다!"
    ]
}

fine_tuning_df = pd.DataFrame(fine_tuning_data)
print(fine_tuning_df)


              input                              output
0  이 제품은 어떻게 사용하나요?          제품 사용법에 대한 정보를 제공해 드리겠습니다.
1      배송이 너무 늦었어요.  배송 지연에 대해 사과드립니다. 빠르게 처리하도록 하겠습니다.
2     이 서비스 정말 좋네요!            고객님께서 만족하셔서 기쁩니다! 감사합니다!


모델 훈련:
모델은 input과 output 데이터를 사용하여 질문에 대한 적절한 답변을 자동으로 생성할 수 있도록 훈련됩니다.

4. 검증 및 피드백
모델이 학습된 후, 실제 메시지와 비교하여 정확도를 검증하고 피드백을 통해 성능을 향상시킵니다.

In [4]:
from sklearn.metrics import accuracy_score

# 예시 예측 결과
true_answers = ["제품 사용법에 대한 정보를 제공해 드리겠습니다.",
                "배송 지연에 대해 사과드립니다. 빠르게 처리하도록 하겠습니다.",
                "고객님께서 만족하셔서 기쁩니다! 감사합니다!"]

predictions = ["제품 사용법에 대한 정보를 제공하겠습니다.",
               "배송 지연에 대해 사과드립니다. 빠르게 처리하도록 하겠습니다.",
               "고객님께서 만족하셔서 기쁩니다! 감사합니다!"]

# 정확도 계산
accuracy = accuracy_score(true_answers, predictions)
print(f"Accuracy: {accuracy * 100}%")


Accuracy: 66.66666666666666%


피드백 수집:
고객 서비스 담당자가 모델의 답변을 확인하고, 잘못된 부분에 대해 피드백을 제공하여 모델을 개선합니다.

전체적인 흐름 요약
데이터 수집: 실제 고객 서비스 메시지 데이터를 수집하여 고객의 요청 유형(질문, 불만, 칭찬)을 분류합니다.
언어 모델 훈련: 수집된 데이터를 기반으로 Fine-tuning을 통해 모델을 훈련시켜 자동 응답 메시지를 생성합니다.
검증 및 피드백: 모델의 정확도를 검증하고, 실시간 피드백을 통해 모델을 지속적으로 개선합니다.

이 과정을 통해 고객 서비스 자동화 시스템을 구축하고, 고객의 메시지에 대해 빠르고 정확한 대응을 할 수 있도록 합니다.